## Generate the QASM Dataset

In [1]:
import os
print(os.getcwd())


d:\course\thesis\Project2\QCD


In [15]:
from algorthims import *
from tqdm import tqdm

In [3]:
def QASM_generator(circuitname, max_qubit):
    # 创建目录
    directory = "Circuits"
    if not os.path.exists(directory):
        os.makedirs(directory)

    
    circuit_func = None
    if circuitname == "grover":
        circuit_func = grover
    elif circuitname == "qft":
        circuit_func = qft
    elif circuitname == "qpe":
        circuit_func = qpe
    elif circuitname == "h_c":
        circuit_func = h_c
    elif circuitname == "rx_c":
        circuit_func = rx_c
    elif circuitname == "rx_gradually_c":
        circuit_func = rx_gradually_c  
    else:
        print("Unsupported circuit name.")
        return

    for n in range(2, max_qubit + 1):
      
        circuit = circuit_func(n)
       
        qasm_str = circuit.qasm()

        filename = os.path.join(directory, f"{circuitname}_{n}.qasm")
        with open(filename, "w") as file:
            file.write(qasm_str)
        print(f"Saved {filename}")

In [2]:
# QASM_generator('qft',20)
# QASM_generator('qpe',20)
# QASM_generator('grover',20)
# QASM_generator('h_c',40)
# QASM_generator('rx_c',40)
# QASM_generator('rx_gradually_c',40)

## Tokenize the Dataset

In [1]:
# Relevant imports

from tokenizers import ByteLevelBPETokenizer
from tokenizers.pre_tokenizers import CharDelimiterSplit
import json
import os
from functools import lru_cache
from typing import TYPE_CHECKING, List, Optional, Tuple
import regex as re
from transformers.tokenization_utils import AddedToken, PreTrainedTokenizer
from transformers.utils import logging
if TYPE_CHECKING:
    from transformers.pipelines.conversational import Conversation

d:\programing\Anaconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\programing\Anaconda\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
d:\programing\Anaconda\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# Defining my own GPT2Tokenizer class to circumvent BPE tokenisation
# Most code is from the Huggingface implementation

logger = logging.get_logger(__name__)

VOCAB_FILES_NAMES = {
    "vocab_file": "vocab.json",
    "merges_file": "merges.txt",
}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "gpt2": "https://huggingface.co/gpt2/resolve/main/vocab.json",
        "gpt2-medium": "https://huggingface.co/gpt2-medium/resolve/main/vocab.json",
        "gpt2-large": "https://huggingface.co/gpt2-large/resolve/main/vocab.json",
        "gpt2-xl": "https://huggingface.co/gpt2-xl/resolve/main/vocab.json",
        "distilgpt2": "https://huggingface.co/distilgpt2/resolve/main/vocab.json",
    },
    "merges_file": {
        "gpt2": "https://huggingface.co/gpt2/resolve/main/merges.txt",
        "gpt2-medium": "https://huggingface.co/gpt2-medium/resolve/main/merges.txt",
        "gpt2-large": "https://huggingface.co/gpt2-large/resolve/main/merges.txt",
        "gpt2-xl": "https://huggingface.co/gpt2-xl/resolve/main/merges.txt",
        "distilgpt2": "https://huggingface.co/distilgpt2/resolve/main/merges.txt",
    },
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "gpt2": 1024,
    "gpt2-medium": 1024,
    "gpt2-large": 1024,
    "gpt2-xl": 1024,
    "distilgpt2": 1024,
}

class GPT2Tokenizer(PreTrainedTokenizer):
    """
    Construct a GPT-2 tokenizer. Based on byte-level Byte-Pair-Encoding.
    This tokenizer has been trained to treat spaces like parts of the tokens (a bit like sentencepiece) so a word will
    be encoded differently whether it is at the beginning of the sentence (without space) or not:
    ```
    >>> from transformers import GPT2Tokenizer
    >>> tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    >>> tokenizer("Hello world")['input_ids']
    [15496, 995]
    >>> tokenizer(" Hello world")['input_ids']
    [18435, 995]
    ```
    You can get around that behavior by passing `add_prefix_space=True` when instantiating this tokenizer or when you
    call it on some text, but since the model was not pretrained this way, it might yield a decrease in performance.
    <Tip>
    When used with `is_split_into_words=True`, this tokenizer will add a space before each word (even the first one).
    </Tip>
    This tokenizer inherits from [`PreTrainedTokenizer`] which contains most of the main methods. Users should refer to
    this superclass for more information regarding those methods.
    Args:
        vocab_file (`str`):
            Path to the vocabulary file.
        merges_file (`str`):
            Path to the merges file.
        errors (`str`, *optional*, defaults to `"replace"`):
            Paradigm to follow when decoding bytes to UTF-8. See
            [bytes.decode](https://docs.python.org/3/library/stdtypes.html#bytes.decode) for more information.
        unk_token (`str`, *optional*, defaults to `<|endoftext|>`):
            The unknown token. A token that is not in the vocabulary cannot be converted to an ID and is set to be this
            token instead.
        bos_token (`str`, *optional*, defaults to `<|endoftext|>`):
            The beginning of sequence token.
        eos_token (`str`, *optional*, defaults to `<|endoftext|>`):
            The end of sequence token.
        add_prefix_space (`bool`, *optional*, defaults to `False`):
            Whether or not to add an initial space to the input. This allows to treat the leading word just as any
            other word. (GPT2 tokenizer detect beginning of words by the preceding space).
    """

    vocab_files_names = VOCAB_FILES_NAMES
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES
    model_input_names = ["input_ids", "attention_mask"]

    def __init__(
        self,
        vocab_file,
        merges_file,
        errors="replace",
        unk_token="<|endoftext|>",
        bos_token="<|endoftext|>",
        eos_token="<|endoftext|>",
        pad_token=None,
        add_prefix_space=False,
        add_bos_token=False,
        **kwargs,
    ):
        bos_token = AddedToken(bos_token, lstrip=False, rstrip=False) if isinstance(bos_token, str) else bos_token
        eos_token = AddedToken(eos_token, lstrip=False, rstrip=False) if isinstance(eos_token, str) else eos_token
        unk_token = AddedToken(unk_token, lstrip=False, rstrip=False) if isinstance(unk_token, str) else unk_token
        pad_token = AddedToken(pad_token, lstrip=False, rstrip=False) if isinstance(pad_token, str) else pad_token
        super().__init__(
            errors=errors,
            unk_token=unk_token,
            bos_token=bos_token,
            eos_token=eos_token,
            pad_token=pad_token,
            add_prefix_space=add_prefix_space,
            add_bos_token=add_bos_token,
            **kwargs,
        )
        self.add_bos_token = add_bos_token

        with open(vocab_file, encoding="utf-8") as vocab_handle:
            self.encoder = json.load(vocab_handle)
        self.decoder = {v: k for k, v in self.encoder.items()}
        self.errors = errors  # how to handle errors in decoding
        self.byte_encoder = self.encoder
        self.byte_decoder = self.decoder
        with open(merges_file, encoding="utf-8") as merges_handle:
            bpe_merges = merges_handle.read().split("\n")[1:-1]
        bpe_merges = [tuple(merge.split()) for merge in bpe_merges]
        self.bpe_ranks = dict(zip(bpe_merges, range(len(bpe_merges))))
        self.cache = {}
        self.add_prefix_space = add_prefix_space

        # Should have added re.IGNORECASE so BPE merges can happen for capitalized versions of contractions
        #self.pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

    @property
    def vocab_size(self):
        return len(self.encoder)

    def get_vocab(self):
        return dict(self.encoder, **self.added_tokens_encoder)

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        if self.add_bos_token:
            bos_token_ids = [self.bos_token_id]
        else:
            bos_token_ids = []

        output = bos_token_ids + token_ids_0

        if token_ids_1 is None:
            return output

        return output + bos_token_ids + token_ids_1

 #   def get_special_tokens_mask(
 #       self, token_ids_0: List[int], token_ids_1: Optional[List[int]] = None, already_has_special_tokens: bool = False
 #   ) -> List[int]:
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer `prepare_for_model` or `encode_plus` methods.
        Args:
            token_ids_0 (`List[int]`):
                List of IDs.
            token_ids_1 (`List[int]`, *optional*):
                Optional second list of IDs for sequence pairs.
            already_has_special_tokens (`bool`, *optional*, defaults to `False`):
                Whether or not the token list is already formatted with special tokens for the model.
        Returns:
            `List[int]`: A list of integers in the range [0, 1]: 1 for a special token, 0 for a sequence token.
        """
 #       if already_has_special_tokens:
 #           return super().get_special_tokens_mask(
 #               token_ids_0=token_ids_0, token_ids_1=token_ids_1, already_has_special_tokens=True
 #           )

  #      if not self.add_bos_token:
   #         return super().get_special_tokens_mask(
    #            token_ids_0=token_ids_0, token_ids_1=token_ids_1, already_has_special_tokens=False
     #       )

      #  if token_ids_1 is None:
       #     return [1] + ([0] * len(token_ids_0))
        #return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1))

    def tokenize_boran(self, text):
        """Tokenize a string."""
        #bpe_tokens = []
        #for token in re.findall(self.pat, text): #This was the huggingface implementation
        #    token = "".join(
        #        self.byte_encoder[b] for b in token.encode("utf-8")
        #    )  # Maps all our bytes to unicode strings, avoiding control tokens of the BPE (spaces in our case)
        #    bpe_tokens.extend(bpe_token for bpe_token in self.bpe(token).split(" "))
        #bpe_token = self.encoder.get(text)
        #bpe_tokens.extend([bpe_token])
        bpe_tokens = text
        try:
          if text[0][0] == '[':
            bpe_tokens = []
            for subtext_index, subtext in enumerate(text):
              text_replaced = text[subtext_index].replace("'", "")
              text_replaced = text_replaced.replace("\\n", "\n")
              text_replaced = text_replaced.replace("\\t", "\t") #new
              text_replaced = text_replaced[1:]
              bpe_tokens.append(text_replaced[:-1].split(', '))

        except:
          bpe_tokens = text

        return bpe_tokens
    
    def tokenize(self, text, is_split_into_words = True):
      bpe_tokens = text
      return text


    def _convert_token_to_id(self, token):
        """Converts a token (str) in an id using the vocab."""
        return self.encoder.get(token, self.encoder.get(self.unk_token))

    def _convert_id_to_token(self, index):
        """Converts an index (integer) in a token (str) using the vocab."""
        return self.decoder.get(index)

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (string) in a single string."""
        text = "".join(tokens)
        text = self.byte_decoder[text]
        return text

    def convert_tokens_to_string(self, tokens: List[str]) -> str:
        return tokens

    def convert_tokens_to_ids(self, tokens):
      return self.encoder.get(tokens, self.encoder.get(self.unk_token))

    def convert_tokens_to_ids(self, tokens):
      try:
        ids = [self.encoder.get(tokens)]
        if tokens == '<pad>' or tokens == '<s>' or tokens == '</s>' or tokens == '<mask>':
          ids = self.encoder.get(tokens)
      except:
        try:
          ids = []
          for token in tokens:
            id = self.encoder.get(token)
            ids.append(id)
        except:
          ids = []
          ids_list = []
          for lists in tokens:
            for token in lists:
              id = self.encoder.get(token)
              ids.append(id)
            ids_list.append(ids)
          ids = ids_list
      return ids

    def save_vocabulary(self, save_directory: str, filename_prefix: Optional[str] = None) -> Tuple[str]:
        if not os.path.isdir(save_directory):
            logger.error(f"Vocabulary path ({save_directory}) should be a directory")
            return
        vocab_file = os.path.join(
            save_directory, (filename_prefix + "-" if filename_prefix else "") + VOCAB_FILES_NAMES["vocab_file"]
        )
        merge_file = os.path.join(
            save_directory, (filename_prefix + "-" if filename_prefix else "") + VOCAB_FILES_NAMES["merges_file"]
        )

        with open(vocab_file, "w", encoding="utf-8") as f:
            f.write(json.dumps(self.encoder, indent=2, sort_keys=True, ensure_ascii=False) + "\n")

        index = 0
        with open(merge_file, "w", encoding="utf-8") as writer:
            writer.write("#version: 0.2\n")
            for bpe_tokens, token_index in sorted(self.bpe_ranks.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        f"Saving vocabulary to {merge_file}: BPE merge indices are not consecutive."
                        " Please check that the tokenizer is not corrupted!"
                    )
                    index = token_index
                writer.write(" ".join(bpe_tokens) + "\n")
                index += 1

        return vocab_file, merge_file

    def prepare_for_tokenization(self, text, is_split_into_words=False, **kwargs):
        add_prefix_space = kwargs.pop("add_prefix_space", self.add_prefix_space)
        if is_split_into_words or add_prefix_space:
            text = text
        return (text, kwargs)

    def _build_conversation_input_ids(self, conversation: "Conversation") -> List[int]:
        input_ids = []
        for is_user, text in conversation.iter_texts():
            input_ids.extend(self.encode(text, add_special_tokens=False) + [self.eos_token_id])
        if len(input_ids) > self.model_max_length:
            input_ids = input_ids[-self.model_max_length :]
        return input_ids
    
    def _batch_encode_plus(self, *args, **kwargs):
      # Custom implementation of _batch_encode_plus
      result = super()._batch_encode_plus(*args, **kwargs)
      # Modify the output format
      new_ids = result["input_ids"]
      new_masks = result["attention_mask"]
      result["input_ids"] = [item for sublist in new_ids for item in sublist]
      result["attention_mask"] = [item for sublist in new_masks for item in sublist]
      if 0 == True:
        result["input_ids"] = result["input_ids"][:3]
        result["attention_mask"] = result["attention_mask"][:3]

      return result

In [ ]:
# Defining the tokenizer
# Minor bug: Prints a number for some reason
with open('gpt_tokenizer/vocab.json', encoding="utf-8") as vocab_handle:
        GPT2Tokenizer.encoder = json.load(vocab_handle)
tokenizer = GPT2Tokenizer.from_pretrained('gpt_tokenizer')
tokenizer.add_special_tokens({
    "eos_token": "</s>",
    "bos_token": "<s>",
    "unk_token": "<unk>",
    "pad_token": "<pad>",
    "mask_token": "<mask>"
})


In [ ]:
# Test to see if our tokenizer works

tokenizer(['h q[0];\n', 'h q[1];\n'])

## Regular Expression

In [9]:
import re

def extract_qasm_info(algorithm, qubit_count):
    """
    Extracts quantum register declarations and quantum gate operations from a QASM file based on the algorithm name and number of qubits.

    Parameters:
    algorithm (str): The name of the algorithm.
    qubit_count (int): The number of qubits.

    Returns:
    None
    """
    # Construct the file path using a raw string
    file_path = fr"D:\course\thesis\Project2\QCD\Circuits\{algorithm}_{qubit_count}.qasm"

    try:
        # Read the content of the file
        with open(file_path, 'r') as file:
            qasm_content = file.read()

        # Regular expressions to match quantum register declarations and quantum gate operations
        qreg_pattern = re.compile(r"qreg\s+\w+\[\d+\];")
        gate_pattern = re.compile(r"\b(h|cx)\b\s+\w+\[\d+\](,\w+\[\d+\])?;")

        # Use the regular expressions to find all matches
        qreg_matches = qreg_pattern.findall(qasm_content)
        gate_matches = gate_pattern.findall(qasm_content)

        # Output the results
        print("Quantum Register Declarations:")
        for match in qreg_matches:
            print(match)

        print("\nQuantum Gate Operations:")
        for match in gate_matches:
            print(match[0])

    except FileNotFoundError:
        print(f"File {algorithm}_{qubit_count}.qasm not found in the specified path.")

# Example usage
extract_qasm_info("qft", 7)


Quantum Register Declarations:
qreg q[7];

Quantum Gate Operations:


In [10]:
def extract_qasm_info(algorithm, qubit_count):
    """
    Extracts quantum register declarations and quantum gate operations from a QASM file based on the algorithm name and number of qubits.

    Parameters:
    algorithm (str): The name of the algorithm.
    qubit_count (int): The number of qubits.

    Returns:
    None
    """
    # Construct the file path using a raw string
    file_path = fr"D:\course\thesis\Project2\QCD\Circuits\{algorithm}_{qubit_count}.qasm"
# Regular expressions to match custom gate applications and h gate operations
    custom_gate_pattern = re.compile(r"\bgate_QFT\b")
    h_gate_pattern = re.compile(r"\bh\s+\w+\[\d+\];")

    # Use the regular expressions to find all matches
    custom_gate_matches = custom_gate_pattern.findall(qasm_content)
    h_gate_matches = h_gate_pattern.findall(qasm_content)

    # Output the results
    print("Custom Gate Applications:")
    for match in custom_gate_matches:
        print(match)

    print("\nHadamard Gate Operations within Custom Gates:")
    for match in h_gate_matches:
        print(match)

SyntaxError: EOF while scanning triple-quoted string literal (2842501474.py, line 40)

In [2]:
from qiskit import QuantumCircuit

# Path to your QASM file
qasm_file_path = 'Circuits\qft_2.qasm'
# Read the QASM file
quantum_circuit = QuantumCircuit.from_qasm_file(qasm_file_path)

# Print the quantum circuit to see its contents
print(quantum_circuit)


d:\programing\Anaconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\programing\Anaconda\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
d:\programing\Anaconda\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


        ┌───────────┐ ░ ┌─┐   
   q_0: ┤0          ├─░─┤M├───
        │  Gate_qft │ ░ └╥┘┌─┐
   q_1: ┤1          ├─░──╫─┤M├
        └───────────┘ ░  ║ └╥┘
   c: 2/═════════════════╬══╬═
                         ║  ║ 
meas: 2/═════════════════╩══╩═
                         0  1 


## genetic programming


In [16]:
from algorthims import *
from tqdm import tqdm

def QASM_generator(circuitname, max_qubit):
    # 创建目录
    directory = "Circuits"
    if not os.path.exists(directory):
        os.makedirs(directory)

    
    circuit_func = None
    if circuitname == "grover":
        circuit_func = grover
    elif circuitname == "qft":
        circuit_func = qft
    elif circuitname == "qpe":
        circuit_func = qpe
    elif circuitname == "h_c":
        circuit_func = h_c
    elif circuitname == "rx_c":
        circuit_func = rx_c
    elif circuitname == "rx_gradually_c":
        circuit_func = rx_gradually_c  
    else:
        print("Unsupported circuit name.")
        return

    for n in range(2, max_qubit + 1):
      
        circuit = circuit_func(n)
       
        qasm_str = circuit.qasm()

        filename = os.path.join(directory, f"{circuitname}_{n}.qasm")
        with open(filename, "w") as file:
            file.write(qasm_str)
        print(f"Saved {filename}")

The basic idea of genetic programming is apply operations which similar to a natural genetic process to a certain task. During each time of a replication(or so-called off-spring), we randomly select some parameters from the parameter space and do some measurement on our selection. The evaluation of these observation is task-dependent. We select some of with the higher perforamnce and keep them to the next generation. we repeat such iteration turn by turns, keeps the highest or some of the highest scores species(the entity of the selection of the parameter sapce) to the  latter off-spring. in the context of circuit decomplier. For a certain quantum circuit $C_A^N$ (A, correspond to the underlying algorithm and N correspond the the scale of the quabit), we want to learn the underlying pattern for such quantum circuit or in other words, let our circyit Decomplier to "explain" the quantum circuit. So the evaluation of each iteration in our genetic programing is how close the circit generated by our decomplier to the ground truth, the actual circuit of the $C_A^N$.

Therefore, to implement the Decomplier using Genetic algorthim, we first just start from the simplest case. We regard every circuit $C_A^N$ can be composed of some combination of the "print" operation and the "For" loop
- For the print operation, the parameter space is composed with the typical syntax in QASM file, it include "Gate Definiation" "register" and "gate operators", to begin from the bottle, we start with[h,q ]

- For the for loop, we need to determine the number of loop we need, we can can struct it as 
```python
for i in range (1,n,1):


In [27]:
import os
import random
from qiskit import QuantumCircuit
import numpy as np
import time

class genetic_Decompiler:
    def __init__(self, algorithm_name, qubit_limit=20, generations=100, pop_size=50, max_length=10, perform_crossover=True, perform_mutation=True, selection_size=1):
        self.algorithm_name = algorithm_name
        self.qubit_limit = qubit_limit
        self.generations = generations
        self.pop_size = pop_size
        self.max_length = max_length
        self.perform_crossover = perform_crossover
        self.perform_mutation = perform_mutation
        self.selection_size = selection_size
        self.gate_operations = ['h', 'rx']  # Presumed set of quantum operations

    def generate_target_circuit_list(self):
        circuit_list = []
        base_path = "Circuits"
        for qubit_number in range(2, 41):
            file_name = f"{self.algorithm_name}_{qubit_number}.qasm"
            file_path = os.path.join(base_path, file_name)
            circuit = QuantumCircuit.from_qasm_file(file_path)
            circuit_list.append(circuit)
        return circuit_list

    def evaluate_decompilation(self, generated_circuit_code, target_circuit, n):
        difference = random.uniform(0, 1)  # Simulated difference score
        return difference

    def generate_initial_population(self):
        population = []
        for _ in range(self.pop_size):
            circuit_length = random.randint(1, self.max_length)
            circuit_code = {'operations': [], 'n': random.randint(2, self.qubit_limit)}
            for i in range(circuit_length):
                operation = random.choice(self.gate_operations)
                qubit_index = f"q[{random.randint(0, circuit_length-1)}]"
                operation_string = f"{operation} {qubit_index};\n"
                if operation == 'rx':
                    phase_n = random.randint(3, self.qubit_limit)
                    phase = f"(pi/2^{phase_n})"
                    operation_string = f"{operation}({phase}) {qubit_index};\n"
                circuit_code['operations'].append(operation_string)
            population.append(circuit_code)
        return population

    def mutate(self, circuit_code, mutation_rate=0.1):
        if random.random() < mutation_rate:
            circuit_length = len(circuit_code['operations'])
            qubit_index = f"q[{random.randint(0, circuit_length-1)}]"
            operation = random.choice(self.gate_operations)
            operation_string = f"{operation} {qubit_index};\n"
            if operation == 'rx':
                phase_n = random.randint(3, self.qubit_limit)
                phase = f"(pi/2^{phase_n})"
                operation_string = f"{operation}({phase}) {qubit_index};\n"
            circuit_code['operations'].append(operation_string)
        return circuit_code

    def crossover(self, circuit1, circuit2):
        crossover_point = len(circuit1['operations']) // 2
        new_operations = circuit1['operations'][:crossover_point] + circuit2['operations'][crossover_point:]
        return {'operations': new_operations, 'n': random.choice([circuit1['n'], circuit2['n']])}

    def run(self):
        target_circuit_list = self.generate_target_circuit_list()
        population = self.generate_initial_population()
        for generation in range(self.generations):
            start_time = time.time()
            fitness_scores = []
            for circuit_code in population:
                total_difference = 0
                for target_circuit in target_circuit_list:
                    total_difference += self.evaluate_decompilation(circuit_code, target_circuit, circuit_code['n'])
                average_difference = total_difference / len(target_circuit_list)
                fitness_scores.append(1 / (1 + average_difference))

            sorted_population = [x for _, x in sorted(zip(fitness_scores, population), key=lambda pair: pair[0], reverse=True)]
            next_generation = sorted_population[:self.selection_size]

            offspring = []
            while len(offspring) < self.pop_size - len(next_generation):
                if self.perform_crossover:
                    parent1, parent2 = random.sample(next_generation, 2)
                    child = self.crossover(parent1, parent2)
                else:
                    child = random.choice(next_generation)
                if self.perform_mutation:
                    child = self.mutate(child)
                offspring.append(child)

            population = next_generation + offspring
            end_time = time.time()
            time_taken = end_time - start_time
            print(f"Generation {generation + 1}/{self.generations} completed in {time_taken:.2f} seconds")

        final_scores = []
        for circuit_code in population:
            total_difference = 0
            for target_circuit in target_circuit_list:
                total_difference += self.evaluate_decompilation(circuit_code, target_circuit, circuit_code['n'])
            average_difference = total_difference / len(target_circuit_list)
            final_scores.append(1 / (1 + average_difference))

        best_code = population[np.argmax(final_scores)]
        return best_code


In [28]:
Decompiler=genetic_Decompiler(algorithm_name='h_c',qubit_limit=20,perform_crossover=False,perform_mutation=False)
Decompiler.run()

Generation 1/100 completed in 0.00 seconds
Generation 2/100 completed in 0.00 seconds
Generation 3/100 completed in 0.00 seconds
Generation 4/100 completed in 0.00 seconds
Generation 5/100 completed in 0.00 seconds
Generation 6/100 completed in 0.00 seconds
Generation 7/100 completed in 0.00 seconds
Generation 8/100 completed in 0.00 seconds
Generation 9/100 completed in 0.00 seconds
Generation 10/100 completed in 0.00 seconds
Generation 11/100 completed in 0.00 seconds
Generation 12/100 completed in 0.01 seconds
Generation 13/100 completed in 0.00 seconds
Generation 14/100 completed in 0.00 seconds
Generation 15/100 completed in 0.00 seconds
Generation 16/100 completed in 0.00 seconds
Generation 17/100 completed in 0.00 seconds
Generation 18/100 completed in 0.00 seconds
Generation 19/100 completed in 0.00 seconds
Generation 20/100 completed in 0.00 seconds
Generation 21/100 completed in 0.00 seconds
Generation 22/100 completed in 0.00 seconds
Generation 23/100 completed in 0.00 secon

{'operations': ['h q[6];\n',
  'rx((pi/2^7)) q[5];\n',
  'rx((pi/2^14)) q[2];\n',
  'rx((pi/2^5)) q[5];\n',
  'rx((pi/2^9)) q[0];\n',
  'rx((pi/2^8)) q[2];\n',
  'h q[4];\n',
  'h q[0];\n'],
 'n': 15}

### Atempt


In [31]:
from qiskit import QuantumCircuit, Aer, transpile
from qiskit.quantum_info import Operator

from qiskit import QuantumCircuit

def remove_non_unitary_operations(original_circuit):
    # Create a new quantum circuit with the same number of qubits and classical bits
    new_circuit = QuantumCircuit(original_circuit.num_qubits)
    
    # Iterate over all operations in the original circuit
    for instr, qargs, cargs in original_circuit.data:
        # Check if the operation is unitary (gates are considered unitary)
        if instr.name not in ['measure', 'reset']:
            # If the operation is unitary, add it to the new circuit
            new_circuit.append(instr, qargs, cargs)
    
    return new_circuit


def qasm_to_unitary(circuit):
    
    # Use the Aer's unitary simulator
    simulator = Aer.get_backend('unitary_simulator')
    
    # Transpile the circuit for the simulator
    transpiled_circuit = transpile(circuit, simulator)
    
    # Run the simulation to get the unitary
    job = simulator.run(transpiled_circuit)
    result = job.result()
    
    # Get the unitary matrix from the result
    unitary = result.get_unitary(transpiled_circuit)
    
    return unitary

# Example usage
qasm_path = 'D:/course/thesis/Project2/QCD/Circuits/qft_13.qasm'
original_circuit = QuantumCircuit.from_qasm_file(qasm_path)
unitary_circuit = remove_non_unitary_operations(original_circuit)
unitary_matrix = qasm_to_unitary(unitary_circuit)
print(unitary_matrix)



Operator([[0.01104854+0.00000000e+00j, 0.01104854-1.35305634e-18j,
           0.01104854-1.35305634e-18j, ..., 0.01104854-1.62366761e-17j,
           0.01104854-1.62366761e-17j, 0.01104854-1.75897324e-17j],
          [0.01104854+0.00000000e+00j, 0.01104854+8.47412587e-06j,
           0.01104853+1.69482467e-05j, ..., 0.01104851-2.54223577e-05j,
           0.01104853-1.69482467e-05j, 0.01104854-8.47412587e-06j],
          [0.01104854+0.00000000e+00j, 0.01104853+1.69482467e-05j,
           0.01104849+3.38964536e-05j, ..., 0.01104843-5.08445807e-05j,
           0.01104849-3.38964536e-05j, 0.01104853-1.69482467e-05j],
          ...,
          [0.01104854+0.00000000e+00j, 0.01104851-2.54223577e-05j,
           0.01104843-5.08445807e-05j, ..., 0.01104828+7.62665346e-05j,
           0.01104843+5.08445807e-05j, 0.01104851+2.54223577e-05j],
          [0.01104854+0.00000000e+00j, 0.01104853-1.69482467e-05j,
           0.01104849-3.38964536e-05j, ..., 0.01104843+5.08445807e-05j,
           0.01104

In [9]:
from qiskit import QuantumCircuit

# 创建一个空列表来存放Qasm量子电路
quantum_circuits = []

# 创建几个简单的量子电路作为示例
for i in range(3):
    # 创建一个包含3个量子比特和3个经典比特的量子电路
    qc = QuantumCircuit(3, 3)
    
    # 添加一些量子门操作作为示例
    qc.h(0)  # 将Hadamard门应用于第一个量子比特
    qc.cx(0, 1)  # 添加一个CNOT门，控制量子比特为0，目标量子比特为1
    qc.measure([0, 1, 2], [0, 1, 2])  # 测量所有量子比特，并将结果存储到所有经典比特
    
    # 将创建的量子电路添加到列表中
    quantum_circuits.append(qc)

# 打印出列表中的每一个量子电路来验证
for i, qc in enumerate(quantum_circuits):
    print(f"Circuit {i}:\n")
    print(qc.draw())  # 使用.draw()方法来可视化电路


Circuit 0:

     ┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
      ┌─┐ └───┘ ║ └╥┘
q_2: ─┤M├───────╫──╫─
      └╥┘       ║  ║ 
c: 3/══╩════════╩══╩═
       2        0  1 
Circuit 1:

     ┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
      ┌─┐ └───┘ ║ └╥┘
q_2: ─┤M├───────╫──╫─
      └╥┘       ║  ║ 
c: 3/══╩════════╩══╩═
       2        0  1 
Circuit 2:

     ┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
      ┌─┐ └───┘ ║ └╥┘
q_2: ─┤M├───────╫──╫─
      └╥┘       ║  ║ 
c: 3/══╩════════╩══╩═
       2        0  1 
